In [ ]:
from google.colab import drive
drive.mount('/content/drive')
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from numpy import mean
from numpy import std
import numpy as np
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from sklearn.metrics import balanced_accuracy_score
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import balanced_accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.model_selection import KFold, cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import ParameterGrid
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
from tensorflow import keras
import os
import tempfile
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.ensemble import HistGradientBoostingClassifier



# Help

In [ ]:
def get_features(data):
  features = list(data)
  features.remove('label')
  return features

def onehot_encode(data_train, data_test):
  X_train = data_train.loc[:, data_train.columns != 'label']
  train_test_features = pd.concat([X_train, data_test])
  to_encode = []
  for c in get_features(data_train):
      if train_test_features[c].dtypes==object:
        to_encode.append(c)
  train_test_features = pd.get_dummies(train_test_features, columns = to_encode)
  train = train_test_features.head(len(data_train))
  train['label']=data_train['label'].values
  test = train_test_features.tail(len(data_test))
  return train, test
def ord_encode(data_train, data_test):
  X_train = data_train.loc[:, data_train.columns != 'label']
  train_test_features = pd.concat([X_train, data_test])
  to_encode = []
  for c in get_features(data_train):
      if train_test_features[c].dtypes==object:
        train_test_features[c]= train_test_features[c].map({'low':0.1, 'medium':0.5, 'high':0.9})
  train = train_test_features.head(len(data_train))
  train['label']=data_train['label'].values
  test = train_test_features.tail(len(data_test))
  return train, test

def feature_selection_var_based(train_data, test_data, frac_to_throw):
  
  vars = []
  for c in get_features(train_data):
    vars.append(train_data[c].var())
  vars.sort()
  
  th = vars[int(len(vars) * frac_to_throw)]
  print(th)
  to_throw = []
  for c in get_features(train_data):
    if train_data[c].var()<=th:
      to_throw.append(c)
  print('to throw for', frac_to_throw)
  print(len(to_throw))
  print(to_throw)  
  return train_data.drop(to_throw, axis=1), test_data.drop(to_throw, axis=1)


def median_imputing_and_scaling(train_data, test_data):
  y = list(train_data['label'])
  features = get_features(train_data)
  train_test_features = pd.concat([train_data[features], test_data[features]])
  tmp = train_test_features.loc[:,features].values
  tmp = SimpleImputer(strategy='median').fit_transform(tmp)
  tmp = StandardScaler().fit_transform(tmp) 
  train_test_features[features]=tmp

  train = train_test_features.head(len(train_data))
  train['label']=train_data['label'].values
  test = train_test_features.tail(len(test_data))
  return train, test

def feature_selection(X_train, X_test, y_train, kv):
  selection = SelectKBest(score_func=mutual_info_classif, k=kv)
  X_train = selection.fit_transform(X_train, y_train) 
  X_test = selection.transform(X_test)
  print("feature sel : k=", kv)
  return X_train, X_test

def sample_df(input_df, rs):
  train_df, val_df = train_test_split(input_df, test_size=0.1, random_state=rs)

  y_train = np.array(train_df.pop('label'))
  y_val = np.array(val_df.pop('label'))

  X_train = np.array(train_df)
  X_val = np.array(val_df)

  return X_train, X_val, y_train, y_val

def oversample(X,y, f=1):
  is_pos = y == 1
  X_pos = X[is_pos]
  X_neg = X[~is_pos]

  y_pos = y[is_pos]
  y_neg = y[~is_pos]

  ids = np.arange(len(X_pos))
  choices = np.random.choice(ids,int(f*len(X_neg)))

  X_pos = X_pos[choices]
  y_pos = y_pos[choices]

  X = np.concatenate([X_pos, X_neg], axis=0)
  y = np.concatenate([y_pos, y_neg], axis=0)

  order = np.arange(len(X))
  np.random.shuffle(order)
  X = X[order]
  y = y[order]
  return X,y



# Preprocessing

In [ ]:
df_train = pd.read_csv('/content/drive/MyDrive/a5/ML-A5-2022_train.csv', sep=',')
df_test = pd.read_csv('/content/drive/MyDrive/a5/ML-A5-2022_test.csv', sep=',')
df_train_enc, df_test_enc = onehot_encode(df_train, df_test) # saved in *-enc.csv
df_train_35, df_test_35 = feature_selection_var_based(df_train_enc, df_test_enc, 0.35)
df_train_fill, df_test_fill = median_imputing_and_scaling(df_train_35, df_test_35)
#df_test_fill.to_csv('/content/drive/MyDrive/a5/ML-A5-2022_test-full-35-median.csv')
#df_train_fill.to_csv('/content/drive/MyDrive/a5/ML-A5-2022_train-full-35-median.csv')

# Load final data

In [ ]:
itest_df = pd.read_csv('/content/drive/MyDrive/a5/ML-A5-2022_test.csv', index_col=0)


In [ ]:
itest_df.head()

In [ ]:
raw_df = pd.read_csv('/content/drive/MyDrive/a5/ML-A5-2022_train-full-35-median.csv', index_col=0)
#test_df = pd.read_csv('/content/drive/MyDrive/a5/ML-A5-2022_test-full-35-median.csv', index_col=0)
#raw_df.head()
#iraw_df = pd.read_csv('/content/drive/MyDrive/a5/ML-A5-2022_train.csv', index_col=0)


In [ ]:
iraw_df.head()

In [ ]:
raw_df.head()

In [ ]:
col_train = list(raw_df.columns)
col_train.remove('label')

col_test = list(test_df.columns)


In [ ]:
for c in col_train:
  if c not in col_test:
    print(c)

In [ ]:
  col_train

In [ ]:
train_df, val_df = train_test_split(raw_df, test_size=0.1, random_state=1)

In [ ]:
y_train = np.array(train_df.pop("label"))
X_train = np.array(train_df)
y_test = np.array(val_df.pop("label"))
X_test = np.array(val_df)

In [ ]:
X_train, X_test = feature_selection(X_train, X_test, y_train, 250)  
X_train, y_train = oversample(X_train, y_train)

clf = HistGradientBoostingClassifier(max_depth=3, max_leaf_nodes=20,
                            min_samples_leaf=25, random_state=42,
                            scoring='balanced_accuracy', max_iter=200)
clf.fit(X_train, y_train)
ypred = clf.predict(X_test)
balanced_accuracy_score(y_test, ypred)

# Evaluation

In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier
# split :
# 80-20 : 77, 78, 78
# 90-10 : 82, 79, 81
# 95-5 : 76, 80; 79.5; 81
# 97.5-2.5
# ~77 % bcr estimation
k=250
bcr1=0
bcr2 = 0
models = []
for f in (0.05, 0.1, 0.20, 0.30, 0.40):
  print(f)
  bcr1 = 0
  bcr2 = 0
  for i in range(10):
    print(i)
    X_train, X_val, y_train, y_val = sample_df(raw_df, i)
    X_train, X_val = feature_selection(X_train, X_val, y_train, k)  
    X_train, y_train = oversample(X_train, y_train)

    clf = HistGradientBoostingClassifier(max_depth=3, max_leaf_nodes=20,
                                min_samples_leaf=25, random_state=42,
                                scoring='balanced_accuracy', max_iter=200)


    
    clf.fit(X_train, y_train)
    ypred = clf.predict(X_val)
    print(balanced_accuracy_score(y_val, ypred))
    bcr1+=balanced_accuracy_score(y_val, ypred)
    print()
  print(bcr2/10)
  print(bcr1/10)
  bcr2/=10
  bcr1/=10


In [ ]:
# majority vote among different models

In [ ]:
test[get_features(test)]
test['label']

In [ ]:
X_train, X_val, y_train, y_val = sample_df(train, 6)
X_train, X_val = feature_selection(X_train, test[get_features(test)], y_train, k)  
X_train, y_train = oversample(X_train, y_train)

clf = HistGradientBoostingClassifier(max_depth=3, max_leaf_nodes=20,
                              min_samples_leaf=25, random_state=42,
                              scoring='balanced_accuracy')
  
clf.fit(X_train, y_train)
balanced_accuracy_score(test['label'], clf.predict(X_val))

In [ ]:
balanced_accuracy_score(test['label'], clf.predict(X_val))

In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier

# ~77 % bcr estimation
k=250
bcr = 0
for i in range(5):
  print(i)
  X_train, X_val, y_train, y_val = sample_df(raw_df, 1)
  X_train, X_val = feature_selection(X_train, X_val, y_train, k)  
  X_train, y_train = oversample(X_train, y_train)

  clf = HistGradientBoostingClassifier(max_depth=3, max_leaf_nodes=20,
                               min_samples_leaf=25, random_state=42,
                               scoring='balanced_accuracy') 
  clf.fit(X_train, y_train)
  ypred = clf.predict(X_val)
  print(balanced_accuracy_score(y_val, ypred))
  bcr+=balanced_accuracy_score(y_val, ypred)
print(bcr/5)
bcr/=5


# Prediction

In [ ]:
raw_df.head()

In [ ]:
test_df = pd.read_csv('/content/drive/MyDrive/a5/ML-A5-2022_test-full-35-median.csv', index_col=0)
test_df.head()

In [ ]:
test_df

In [ ]:
X_test = np.array(test_df)
X_train, _, y_train, _ = sample_df(raw_df, i)
print(X_train.shape)
X_train, X_test = feature_selection(X_train, X_test, y_train, 250)  
X_train, y_train = oversample(X_train, y_train)

clf = HistGradientBoostingClassifier(max_depth=3, max_leaf_nodes=20,
                               min_samples_leaf=25, random_state=42,
                               scoring='balanced_accuracy') 
clf.fit(X_train, y_train)
ypred = clf.predict(X_test)
np.count_nonzero(ypred == 1)
test_df['Prediction']=ypred
pred_df = test_df['Prediction']
pred_df.to_csv('/content/drive/MyDrive/a5/histPrediction.csv')
pred_df.to_csv('histPredictionbr.csv')

In [ ]:
X_train.shape